In [ ]:
import pytest
import os
import json
from unittest.mock import patch, MagicMock
from rss_collector.orchestration import process_feeds

# Mock Fixtures
@pytest.fixture
def mock_feed_data():
    return {
        "entries": [
            {
                "title": "Article 1",
                "link": "http://example.com/article1",
                "published": "2024-11-30T12:00:00Z",
            },
            {
                "title": "Article 2",
                "link": "http://example.com/article2",
                "published": "2024-11-30T13:00:00Z",
            },
        ]
    }

@pytest.fixture
def mock_proxy_settings():
    return {
        "app_proxy": "http://mock_proxy.com:85",
        "username": "mock_user",
        "password": "mock_password",
    }

@pytest.fixture
def test_feed_urls():
    return [
        "http://mock_feed_1.com/rss",
        "http://mock_feed_2.com/rss",
    ]

# Tests
@patch("rss_collector.orchestration.get_proxy_settings")
def test_proxy_settings(mock_get_proxy_settings, mock_proxy_settings):
    mock_get_proxy_settings.return_value = mock_proxy_settings
    proxy_settings = mock_get_proxy_settings()
    assert proxy_settings == mock_proxy_settings
    mock_get_proxy_settings.assert_called_once()

@patch("rss_collector.orchestration.fetch_feed_content_with_proxy")
def test_feed_urls(mock_fetch_feed, test_feed_urls, mock_proxy_settings):
    mock_fetch_feed.return_value = "Mock RSS Feed Content"
    for feed_url in test_feed_urls:
        feed_content = mock_fetch_feed(feed_url, **mock_proxy_settings)
        assert feed_content == "Mock RSS Feed Content"
    assert mock_fetch_feed.call_count == len(test_feed_urls)

@patch("rss_collector.orchestration.parse_feed")
def test_feed_data(mock_parse_feed, mock_feed_data):
    mock_parse_feed.return_value = mock_feed_data
    feed_content = "Mock RSS Feed Content"
    parsed_feed = mock_parse_feed(feed_content)
    assert parsed_feed == mock_feed_data
    mock_parse_feed.assert_called_once_with(feed_content)

@patch("rss_collector.orchestration.get_proxy_settings")
@patch("rss_collector.orchestration.fetch_feed_content_with_proxy")
@patch("rss_collector.orchestration.parse_feed")
def test_process_feeds(
    mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings, test_feed_urls
):
    # Mock Dependencies
    mock_get_proxy_settings.return_value = mock_proxy_settings
    mock_fetch_feed.return_value = "Mock RSS Feed Content"
    mock_parse_feed.return_value = mock_feed_data

    # Run process_feeds
    process_feeds(test_feed_urls)

    # Check output
    output_dir = "rss_metadata"
    assert os.path.exists(output_dir), "Output directory was not created."

    saved_files = os.listdir(output_dir)
    assert len(saved_files) == 1, f"Expected one metadata file, but found {len(saved_files)}."

    # Validate metadata content
    metadata_file = os.path.join(output_dir, saved_files[0])
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    assert "timestamp" in metadata
    assert "articles" in metadata
    assert len(metadata["articles"]) == len(mock_feed_data["entries"])

    # Cleanup
    for file in saved_files:
        os.remove(os.path.join(output_dir, file))
    os.rmdir(output_dir)

    # Assert calls
    mock_get_proxy_settings.assert_called_once()
    assert mock_fetch_feed.call_count == len(test_feed_urls)
    assert mock_parse_feed.call_count == len(test_feed_urls)
